In this Jupyter Notebook we run the experiments of Section 6.3.

First of all, we have to import the required libraries as well as the data

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.autograd import grad
import numpy as np
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
import random
from CF_NeuralNetwork import FFNN
from CF_NeuralNetwork_PC import FFNNPC

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transforms.ToTensor(),
                                        download=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transforms.ToTensor(),
                                       download=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, 
                                         shuffle=False, num_workers=2)

X_train=trainset.data
Y_train=trainset.targets
X_test=testset.data
Y_test=testset.targets
Y_train_alt=np.array(Y_train)
X_train=X_train/X_train.sum(axis=(1,2,3))[:,None,None,None]*32*32
X_train=np.reshape(X_train,(np.size(X_train,0),32*32*3))
X_reference=X_train[34]
X_reference=np.reshape(X_reference,32*32*3)
X_test=X_test/X_test.sum(axis=(1,2,3))[:,None,None,None]*32*32
X_test=np.reshape(X_test,(np.size(X_test,0),32*32*3))
Y_train=np.loadtxt('CIFAR_D_train', delimiter=',')
Y_test=np.loadtxt('CIFAR_D_test', delimiter=',')
X_train=np.delete(X_train,34,0)
Y_train=np.delete(Y_train,34,0)


# Transform data to torch variables
x_Train_t=Variable(torch.from_numpy(X_train).float(), requires_grad=True)
y_Train_t=Variable(torch.from_numpy(Y_train).float(), requires_grad=False)
x_Test_t=Variable(torch.from_numpy(X_test).float(), requires_grad=False)
y_Test_t=Variable(torch.from_numpy(Y_test).float(), requires_grad=False)

def my_rel_loss(output,target):
    loss=torch.abs(output-target)
    loss=torch.div(loss,target)
    loss=torch.mean(loss)
    return loss

def my_abs_loss(output,target):
    loss=torch.abs(output-target)
    loss=torch.mean(loss)
    return loss.reshape(-1)

Experiment 6.6: We consider the same neural network architecture as in Experiment 6.5. However, in contrast to the experiments before, we allow all the neural network parameters to be trainable, including the ones from the part corresponding
to the representation of the maxima function. Nonetheless, we initialise the second part of the architecture to represent the maxima function. We start by randomly initialising the first hidden layer. 

In [ ]:
# Initialize the neural network
dimension_space=x_Train_t.size(dim=1)
number_potentials=2**12
model=FFNN(dimension_space,number_potentials,False)
optimizer = torch.optim.Adam(model.parameters())
epochs=300
batch_size = 64

# Training Loss
epoch_Loss=[]
# List for the epochs
epoch_list=[]
# Test Loss
epoch_Loss_Val=[]

for epochc in range(epochs):
    permutation = torch.randperm(x_Train_t.size()[0])
    acc_loss=0
    counter=0
    for i in range(0,x_Train_t.size()[0], batch_size):
        optimizer.zero_grad()
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = x_Train_t[indices], y_Train_t[indices]
        y_pred=model(batch_x)
        y_pred=y_pred.reshape(-1)
        loss=my_rel_loss(y_pred,batch_y)
        loss.backward()
        optimizer.step()
        acc_loss += loss
        counter += 1
    loss_app=acc_loss/counter
    y_pred_val=model(x_Test_t)
    y_pred_val=y_pred_val.reshape(-1)
    loss_Val=my_rel_loss(y_pred_val,y_Test_t)
    epoch_Loss_Val.append(loss_Val.detach().numpy())
    epoch_Loss.append(loss_app.detach().numpy())
    epoch_list.append(epochc+1)

# Create the numpy arrays for the plots
counter=np.array(epoch_list)
mse_loss=np.array(epoch_Loss)
mse_loss_val=np.array(epoch_Loss_Val)

# Generate the plots
fig, ax = plt.subplots()
ax.set_ylim([0.001, 1])
plt.xlabel('number of epochs',fontsize=16)
plt.ylabel('mean relative error',fontsize=16)
plot_1a, =ax.semilogy(counter,mse_loss,color='blue',label='training set')
plot_1b, =ax.semilogy(counter,mse_loss_val,'--',color='green',label='test set')
plt.axhline(y = 0.037183734101399565, color = 'r', linestyle = ':')
ax.legend(handles=[plot_1a,plot_1b],loc='upper right',fontsize=16)
plt.yticks(fontsize=16)
plt.xticks(fontsize=16)
plt.grid()
plt.grid(which='minor',alpha=0.2)
plt.show()

Next, we use the precomputed Kantorovich potentials to initialise the first hidden layer.

In [ ]:
B0f=np.loadtxt('CIFAR_P_f.gz', delimiter=',')
B0g=np.loadtxt('CIFAR_P_g.gz', delimiter=',')
bvectot=np.matmul(B0g,X_reference)
idx=np.arange(B0f.shape[0])
np.random.shuffle(idx)
BS=B0f[idx,:]
bvecS=bvectot[idx]
dimension_space=x_Train_t.size(dim=1)
number_potentials=2**12
Amat=BS[:number_potentials,:]
bvec=bvecS[0:number_potentials]

model=FFNNPC(Amat,bvec,dimension_space,number_potentials,False)
optimizer = torch.optim.Adam(model.parameters())
epochs = 300
batch_size = 64
# Training Loss
epoch_Loss=[]
# List for the epochs
epoch_list=[]
# Test Loss
epoch_Loss_Val=[]

# Initial erros; i.e., based on the pre-computed potentials, but without any training
y_pred_train=model(x_Train_t)
y_pred_train=y_pred_train.reshape(-1)
loss_train=my_rel_loss(y_pred_train,y_Train_t)
print(loss_train)
y_pred_test=model(x_Test_t)
y_pred_test=y_pred_test.reshape(-1)
loss_test=my_rel_loss(y_pred_test,y_Test_t)
epoch_Loss_Val.append(loss_test.detach().numpy())
epoch_Loss.append(loss_train.detach().numpy())
epoch_list.append(0)

for epochc in range(epochs):
    permutation = torch.randperm(x_Train_t.size()[0])
    acc_loss=0
    counter=0
    for i in range(0,x_Train_t.size()[0], batch_size):
        optimizer.zero_grad()
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = x_Train_t[indices], y_Train_t[indices]
        y_pred=model(batch_x)
        y_pred=y_pred.reshape(-1)
        loss=my_rel_loss(y_pred,batch_y)
        loss.backward()
        optimizer.step()
        acc_loss += loss
        counter += 1
    loss_app=acc_loss/counter
    y_pred_val=model(x_Test_t)
    y_pred_val=y_pred_val.reshape(-1)
    loss_Val=my_rel_loss(y_pred_val,y_Test_t)
    epoch_Loss_Val.append(loss_Val.detach().numpy())
    epoch_Loss.append(loss_app.detach().numpy())
    epoch_list.append(epochc+1)

# Create numpy arrays for the plots
counter=np.array(epoch_list)
mse_loss=np.array(epoch_Loss)
mse_loss_val=np.array(epoch_Loss_Val)


# Generate the plots
fig, ax = plt.subplots()
ax.set_ylim([0.001, 1])
plt.xlabel('number of epochs',fontsize=16)
plt.ylabel('mean relative error',fontsize=16)
plot_1a, =ax.semilogy(counter,mse_loss,color='blue',label='training set')
plot_1b, =ax.semilogy(counter,mse_loss_val,'--',color='green',label='test set')
plt.axhline(y = 0.037183734101399565, color = 'r', linestyle = ':')
ax.legend(handles=[plot_1a,plot_1b],loc='upper right',fontsize=16)
plt.yticks(fontsize=16)
plt.xticks(fontsize=16)
plt.grid()
plt.grid(which='minor',alpha=0.2)
plt.show()

Next, we shall prepare the discretised $H^{1,2}$-norm, which will be employed in the Experiment 6.7 below.

In [ ]:
def nth_derivative(f, wrt, n):
    for i in range(n):
        grads = grad(f.sum(), wrt, create_graph=True, allow_unused=True)[0]
    return grads

def phigradx(model):
    a=model.layers[0].weight.data
    f=a.size(dim=0)
    a=a.reshape(f,32,32,3)
    b=torch.zeros(f,32,32,3)
    b[:,0:-1,:,:]=a[:,1:,:,:]-a[:,0:-1,:,:]
    b[:,-1,:,:]=a[:,-1,:,:]-a[:,-2,:,:]
    c=b.reshape(f,32*32*3)
    return c

def phigrady(model):
    a=model.layers[0].weight.data
    f=a.size(dim=0)
    a=a.reshape(f,32,32,3)
    b=torch.zeros(f,32,32,3)
    b[:,:,0:-1,:]=a[:,:,1:,:]-a[:,:,0:-1,:]
    b[:,:,-1,:]=a[:,:,-1,:]-a[:,:,-2,:]
    c=b.reshape(f,32*32*3)
    return c

def phigradRGB(model):
    a=model.layers[0].weight.data
    f=a.size(dim=0)
    a=a.reshape(f,32,32,3)
    b=torch.zeros(f,32,32,3)
    b[:,:,:,0:-1]=a[:,:,:,1:]-a[:,:,:,0:-1]
    b[:,:,:,-1]=a[:,:,:,-1]-a[:,:,:,-2]
    c=b.reshape(f,32*32*3)
    return c

def H12norm(inp,model):
    dfs=0.0
    fs=0.0
    pred=model(inp)
    ts=pred.size(dim=0)
    gpx=phigradx(model)
    gpy=phigrady(model)
    gpz=phigradRGB(model)
    inp2=model.modelPhi(inp)
    Psi=model.modelPsi(inp2)
    dPsi=nth_derivative(Psi,wrt=inp2,n=1)
    Dvecx=torch.mm(dPsi,gpx)
    Dvecy=torch.mm(dPsi,gpy)
    Dvecz=torch.mm(dPsi,gpz)
    Dvecxsq=torch.square(Dvecx)
    Dvecysq=torch.square(Dvecy)
    Dveczsq=torch.square(Dvecz)
    Dvecsq=Dvecxsq+Dvecysq+Dveczsq
    dfsvec=torch.mul(Dvecsq,inp)
    dfs=torch.sum(dfsvec)
    dfs=1/ts*dfs
    predt=torch.transpose(pred,0,1)
    fs=1/ts*torch.mm(predt,pred)
    res=torch.sqrt(dfs+fs)
    return res.reshape(-1)

Experiment 6.7: We repeat the experiment from before, but add a regularisation by means of the discretised $H^{1,2}$-norm. First, the regularisation parameter is set to $\lambda=0.1$.

In [ ]:
# Initialize the neural network
dimension_space=x_Train_t.size(dim=1)
number_potentials=2**10
model=FFNN(dimension_space,number_potentials,False)
optimizer = torch.optim.Adam(model.parameters())
epochs=300
batch_size = 64
lambdas = 0.1

# Training Loss
epoch_Loss=[]
# List for the epochs
epoch_list=[]
# Test Loss
epoch_Loss_Val=[]

for epochc in range(epochs):
    permutation = torch.randperm(x_Train_t.size()[0])
    acc_loss=0
    counter=0
    for i in range(0,x_Train_t.size()[0], batch_size):
        optimizer.zero_grad()
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = x_Train_t[indices], y_Train_t[indices]
        y_pred=model(batch_x)
        y_pred=y_pred.reshape(-1)
        loss=my_abs_loss(y_pred,batch_y)
        h12n=H12norm(batch_x,model)
        lossO=loss+lambdas*H12norm(batch_x,model)
        lossO.backward()
        optimizer.step()
        acc_loss += my_rel_loss(y_pred,batch_y)
        counter += 1
    loss_app=acc_loss/counter
    y_pred_val=model(x_Test_t)
    y_pred_val=y_pred_val.reshape(-1)
    loss_Val=my_rel_loss(y_pred_val,y_Test_t)
    epoch_Loss_Val.append(loss_Val.detach().numpy())
    epoch_Loss.append(loss_app.detach().numpy())
    epoch_list.append(epochc+1)

# Create the numpy arrays for the plots
counter=np.array(epoch_list)
mse_loss=np.array(epoch_Loss)
mse_loss_val=np.array(epoch_Loss_Val)

# Generate the plots
fig, ax = plt.subplots()
ax.set_ylim([0.001, 1])
plt.xlabel('number of epochs',fontsize=16)
plt.ylabel('mean relative error',fontsize=16)
plot_1a, =ax.semilogy(counter,mse_loss,color='blue',label='training set')
plot_1b, =ax.semilogy(counter,mse_loss_val,'--',color='green',label='test set')
plt.axhline(y = 0.037183734101399565, color = 'r', linestyle = ':')
ax.legend(handles=[plot_1a,plot_1b],loc='upper right',fontsize=16)
plt.yticks(fontsize=16)
plt.xticks(fontsize=16)
plt.grid()
plt.grid(which='minor',alpha=0.2)
plt.show()

Next, the regularisation parameter is set to be $\lambda=0.001$.

In [ ]:
# Initialize the neural network
dimension_space=x_Train_t.size(dim=1)
number_potentials=2**10
model=FFNN(dimension_space,number_potentials,False)
optimizer = torch.optim.Adam(model.parameters())
epochs=300
batch_size = 64
lambdas = 0.001

# Training Loss
epoch_Loss=[]
# List for the epochs
epoch_list=[]
# Test Loss
epoch_Loss_Val=[]

for epochc in range(epochs):
    permutation = torch.randperm(x_Train_t.size()[0])
    acc_loss=0
    counter=0
    for i in range(0,x_Train_t.size()[0], batch_size):
        optimizer.zero_grad()
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = x_Train_t[indices], y_Train_t[indices]
        y_pred=model(batch_x)
        y_pred=y_pred.reshape(-1)
        loss=my_abs_loss(y_pred,batch_y)
        h12n=H12norm(batch_x,model)
        lossO=loss+lambdas*H12norm(batch_x,model)
        lossO.backward()
        optimizer.step()
        acc_loss += my_rel_loss(y_pred,batch_y)
        counter += 1
    loss_app=acc_loss/counter
    y_pred_val=model(x_Test_t)
    y_pred_val=y_pred_val.reshape(-1)
    loss_Val=my_rel_loss(y_pred_val,y_Test_t)
    epoch_Loss_Val.append(loss_Val.detach().numpy())
    epoch_Loss.append(loss_app.detach().numpy())
    epoch_list.append(epochc+1)

# Create the numpy arrays for the plots
counter=np.array(epoch_list)
mse_loss=np.array(epoch_Loss)
mse_loss_val=np.array(epoch_Loss_Val)

# Generate the plots
fig, ax = plt.subplots()
ax.set_ylim([0.001, 1])
plt.xlabel('number of epochs',fontsize=16)
plt.ylabel('mean relative error',fontsize=16)
plot_1a, =ax.semilogy(counter,mse_loss,color='blue',label='training set')
plot_1b, =ax.semilogy(counter,mse_loss_val,'--',color='green',label='test set')
plt.axhline(y = 0.037183734101399565, color = 'r', linestyle = ':')
ax.legend(handles=[plot_1a,plot_1b],loc='upper right',fontsize=16)
plt.yticks(fontsize=16)
plt.xticks(fontsize=16)
plt.grid()
plt.grid(which='minor',alpha=0.2)
plt.show()

In Experiment 6.8, we compare the performance of our neural network archictecture from Experiment 6.6 against a convolutional neural network (CNN). The architecture of the CNN is borrowed, up to some modification, from https://www.kaggle.com/code/shadabhussain/cifar-10-cnn-using-pytorch.

In [ ]:
from CF_CNN import Cifar10Model

model = Cifar10Model()

# Initialize the neural network
optimizer = torch.optim.Adam(model.parameters())
epochs = 300
batch_size = 64

# Training Loss
epoch_Loss=[]
# List for the epochs
epoch_list=[]
# Test Loss
epoch_Loss_Val=[]

for epoch in range(epochs):
    acc_loss = 0
    counter = 0
    
    # Shuffle the dataset manually
    permutation = torch.randperm(x_Train_t.size()[0])
    
    for i in range(0, x_Train_t.size(0), batch_size):
        optimizer.zero_grad()
        # Batch selection
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = x_Train_t[indices], y_Train_t[indices]
        # Forward pass
        y_pred = model(batch_x)
        y_pred = y_pred.reshape(-1)
        
        # Compute loss
        loss = my_rel_loss(y_pred, batch_y)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        acc_loss += loss
        counter += 1

    # Calculate average loss
    loss_app=acc_loss/counter
    print(loss_app)
    y_pred_val=model(x_Test_t)
    y_pred_val=y_pred_val.reshape(-1)
    loss_Val=my_rel_loss(y_pred_val,y_Test_t)
    epoch_Loss_Val.append(loss_Val.detach().numpy())
    epoch_Loss.append(loss_app.detach().numpy())
    epoch_list.append(epoch+1)


# Create the numpy arrays for the plots
counter=np.array(epoch_list)
mse_loss=np.array(epoch_Loss)
mse_loss_val=np.array(epoch_Loss_Val)

# Generate the plots
fig, ax = plt.subplots()
ax.set_ylim([0.001, 1])
plt.xlabel('number of epochs',fontsize=16)
plt.ylabel('mean relative error',fontsize=16)
plot_1a, =ax.semilogy(counter,mse_loss,color='blue',label='training set')
plot_1b, =ax.semilogy(counter,mse_loss_val,'--',color='green',label='test set')
plt.axhline(y = 0.037183734101399565, color = 'r', linestyle = ':')
ax.legend(handles=[plot_1a,plot_1b],loc='upper right',fontsize=16)
plt.yticks(fontsize=16)
plt.xticks(fontsize=16)
plt.grid()
plt.grid(which='minor',alpha=0.2)

plt.show()